In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Conv2DTranspose, Dropout
from tensorflow.keras.layers import MaxPooling2D, UpSampling2D, concatenate, Layer, Dense, Activation
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import Model
from google.colab import drive
import tensorflow as tf
import numpy as np
import datetime
import warnings
import os

#drive.mount('/content/drive')
warnings.filterwarnings("ignore")

### Sub-clases del modelo

In [ ]:
class Incremento(Layer):
    def __init__(self, filtros, kernel_size= 3):
        super(Incremento, self).__init__()
        self.conv1= Conv2D(filtros, kernel_size= kernel_size, padding= "same", activation="relu")
        self.conv2= Conv2D(filtros, kernel_size= kernel_size, padding= "same", activation="relu")
        self.maxpool= MaxPooling2D(3, strides= 2, padding= "same")
        self.bn1= BatchNormalization()
        self.bn2= BatchNormalization()

    def call(self, input_tensor):
        x= self.conv1(input_tensor)        
        x= self.bn1(x)
        x= self.conv2(x)
        x= self.bn2(x)
        resdual= x
        x= self.maxpool(x)
        
        return resdual, x





class Cuello_botella(Layer):
    def __init__(self, filtros_conv, filtros_transpose, kernel_size= 3):
        super(Cuello_botella, self).__init__()
        self.conv1= Conv2D(filtros_conv, kernel_size= kernel_size, padding= "same", activation="relu")
        self.conv2= Conv2D(filtros_conv, kernel_size= kernel_size, padding= "same", activation="relu")
        self.convT= Conv2DTranspose(filtros_transpose, kernel_size= kernel_size, padding= "same", activation= "relu")
        self.bn1= BatchNormalization()
        self.bn2= BatchNormalization()

    def call(self, input_tensor):
        x= self.conv1(input_tensor)
        x= self.bn1(x)
        x= self.conv2(x)
        x= self.bn2(x)
        return self.convT(x)






class Decremento(Layer):
    def __init__(self, filtros, kernel_size= 3):
        super(Decremento, self).__init__()
        self.conv1= Conv2D(filtros, kernel_size= kernel_size, padding= "same", activation="relu")
        self.conv2= Conv2D(filtros, kernel_size= kernel_size, padding= "same", activation="relu")
        self.convT= Conv2DTranspose(int(filtros/2), kernel_size= kernel_size, padding= "same")
        self.bn1= BatchNormalization()
        self.bn2= BatchNormalization()
    
    def call(self, input_tensor):
        x= self.conv1(input_tensor)
        x= self.bn1(x)
        x= self.conv2(x)
        x= self.bn2(x)
        return self.convT(x)





class Crop_and_cat(Layer):
    def __init__(self):
        super(Crop_and_cat, self).__init__()
        self.Upsam= UpSampling2D(2)

    def call(self, input_tensor, res):
        x= self.Upsam(input_tensor)
        return concatenate([x, res])





class Capa_final(Layer):
    def __init__(self, filtros, num_clases, kernel_size= 3):
        super(Capa_final, self).__init__()

        self.conv1= Conv2D(filtros, kernel_size= kernel_size, padding= "same", activation="relu")
        self.conv2= Conv2D(filtros, kernel_size= kernel_size, padding= "same", activation="relu")
        self.conv_f= Conv2D(num_clases, kernel_size= kernel_size, padding= "same", activation="softmax")
        self.bn1= BatchNormalization()
        self.bn2= BatchNormalization()

    def call(self, input_tensor):
        x= self.conv1(input_tensor)
        x= self.bn1(x)
        x= self.conv2(x)
        x= self.bn2(x)
        x= self.conv_f(x)
        return x

### Modulos de atención

In [ ]:
class CBAM(Layer):
  def __init__(self, num_kernels, ratio):
    super(CBAM, self).__init__()

    reduccion= int(num_kernels// ratio)
    self.davr_r= Dense(reduccion, activation= 'relu')
    self.dmax_r= Dense(reduccion, activation= 'relu')

    self.davr_a= Dense(num_kernels, activation= 'relu')
    self.dmax_a= Dense(num_kernels, activation= 'relu')

    self.act= Activation('sigmoid')
    self.act_1= Activation('sigmoid')

    self.conv= Conv2D(1, kernel_size= 7, padding= 'same')
    self.batch= BatchNormalization()

  def call(self, entrada):
    favr= tf.reduce_mean(entrada, axis= [-1], keepdims= True)
    fmax= tf.reduce_max(entrada, axis= [-1], keepdims= True)
    x_1= self.davr_r(favr)
    x_2= self.dmax_r(fmax)
    x= tf.add(self.davr_a(x_1), self.dmax_a(x_2))
    x= self.act(x)
    r= tf.math.multiply(entrada, x)

    favr_s= tf.reduce_mean(r, axis= [-1], keepdims= True)
    fmax_s= tf.reduce_max(r, axis= [-1], keepdims= True)
    r= tf.concat([favr_s, fmax_s], axis= -1)
    r= self.conv(r)
    r= self.batch(r)
    return self.act_1(r)

In [ ]:
class UNET(Model):
  def __init__(self, num_clases):
    super(UNET, self).__init__()

    # Atencion encoder
    self.att1= CBAM(64, 16)
    self.att2= CBAM(128, 16)
    self.att3= CBAM(256, 16)
    self.att4= CBAM(256, 16)

    self.dtt1= CBAM(128, 16)
    self.dtt2= CBAM(64, 16)


    self.convi_1= Incremento(64)
    self.convi_2= Incremento(128)
    self.convi_3= Incremento(256)

    self.bot= Cuello_botella(512, 256)

    self.crop1= Crop_and_cat()
    self.crop2= Crop_and_cat()
    self.crop3= Crop_and_cat()


    self.dec1= Decremento(256)
    self.dec2= Decremento(128)

    self.salida= Capa_final(64, num_clases)
    #self.drop= Dropout(rate= 0.5)


  def call(self, entrada_tam):

    res1, x= self.convi_1(entrada_tam)
    x= self.att1(x)

    res2, x= self.convi_2(x)
    x= self.att2(x)

    res3, x= self.convi_3(x)
    x= self.att3(x)
    
    x= self.bot(x)
    x= self.att4(x)

    x= self.crop1(x, res3)
    x= self.dec1(x)
    x= self.dtt1(x)

    x= self.crop2(x, res2)
    x= self.dec2(x)
    x= self.dtt2(x)

    x= self.crop3(x, res1) 
    x= self.salida(x)
    #x= self.drop(x)

    return x

  def build_graph(self, raw_shape):
    x= tf.keras.layers.Input(shape= raw_shape)
    return tf.keras.Model(inputs=[x], outputs= self.call(x))

In [ ]:
raw_input = (320, 320, 3)

mi_modelo= UNET(2)
y= mi_modelo(tf.ones(shape=(0, *raw_input)))

mi_modelo.build_graph(raw_input).summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 320, 320, 3  0           []                               
                                )]                                                                
                                                                                                  
 incremento (Incremento)        ((None, 320, 320, 6  39232       ['input_1[0][0]']                
                                4),                                                               
                                 (None, 160, 160, 6                                               
                                4))                                                               
                                                                                              

In [ ]:
!unzip "/content/drive/MyDrive/Kika/Base de datos/Ent.zip" -d "/content/sample_data/"
!unzip "/content/drive/MyDrive/Kika/Base de datos/Val.zip" -d "/content/sample_data/"

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: /content/sample_data/Ent/mask_patches/8940.png  
  inflating: /content/sample_data/Ent/mask_patches/8941.png  
  inflating: /content/sample_data/Ent/mask_patches/8942.png  
  inflating: /content/sample_data/Ent/mask_patches/8943.png  
  inflating: /content/sample_data/Ent/mask_patches/8944.png  
  inflating: /content/sample_data/Ent/mask_patches/8945.png  
  inflating: /content/sample_data/Ent/mask_patches/8946.png  
  inflating: /content/sample_data/Ent/mask_patches/8947.png  
  inflating: /content/sample_data/Ent/mask_patches/8948.png  
  inflating: /content/sample_data/Ent/mask_patches/8949.png  
  inflating: /content/sample_data/Ent/mask_patches/895.png  
  inflating: /content/sample_data/Ent/mask_patches/8950.png  
  inflating: /content/sample_data/Ent/mask_patches/8951.png  
  inflating: /content/sample_data/Ent/mask_patches/8952.png  
  inflating: /content/sample_data/Ent/mask_patches/8953.png  
  inflatin

In [ ]:
ent_input_dir= "/content/sample_data/Ent/img_patches"
ent_target_dir= "/content/sample_data/Ent/mask_patches"

val_input_dir= "/content/sample_data/Val/img_patches"
val_target_dir= "/content/sample_data/Val/mask_patches"

input_img_paths = sorted(
    [
        os.path.join(ent_input_dir, fname)
        for fname in os.listdir(ent_input_dir) 
        if fname.endswith(".png")
    ]
)

target_img_paths = sorted(
    [
        os.path.join(ent_target_dir, fname)
        for fname in os.listdir(ent_target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

val_input_paths = sorted(
    [
        os.path.join(val_input_dir, fname)
        for fname in os.listdir(val_input_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

val_target_paths = sorted(
    [
        os.path.join(val_target_dir, fname)
        for fname in os.listdir(val_target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

no_ent= len(input_img_paths)
no_val= len(val_target_paths)

In [ ]:
class MiClasificacion():
    def __init__(self, No_img, cont= 0, batch_size= 32, img_size= (304, 304)):
        self.batch_size= batch_size
        self.img_size= img_size
        self.cont= cont
        self.steps= No_img// self.batch_size
        self.ID_input= np.arange(No_img)

    def dar_datos(self, img_dir, val_dir):
        if self.steps== self.cont or self.cont== 0:
            np.random.shuffle(self.ID_input)
            self.cont= 0
        
        batch_img= []
        batch_tar= []
        i= self.cont* self.batch_size
        self.cont+= 1

        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8") 


        for ig in self.ID_input[i : i + self.batch_size]:
            batch_img.append(img_dir[ig])
            batch_tar.append(val_dir[ig])


        for j, path in enumerate(batch_img):
                img = load_img(path, target_size= self.img_size)
                x[j] = np.array(img)/255.0  

        for j, path in enumerate(batch_tar):
                img = load_img(path,target_size= self.img_size, color_mode="grayscale")
                y[j] = np.expand_dims(img, 2)
                
                for r in range(y.shape[1]):
                    for g in range(y.shape[2]):
                        if y[j, r, g, 0]!= 0:
                            y[j, r, g, 0]= 1
    
        return x, y

In [ ]:
img_size = (320, 320)
num_classes = 2; batch_size_t = 16; batch_size_v = 16

train_gen = MiClasificacion(no_ent, batch_size= batch_size_t, img_size= img_size)
val_gen = MiClasificacion(no_val, batch_size=batch_size_v, img_size= img_size)

In [ ]:
optimizer= tf.keras.optimizers.Adam()
loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits= False)

train_loss= tf.keras.metrics.Mean(name= 'train_loss')
train_accuracy= tf.keras.metrics.SparseCategoricalAccuracy(name= 'train_accuracy')
train_recall= tf.keras.metrics.Recall(name= 'train_recall')
train_precision= tf.keras.metrics.Precision(name= 'train_precision')


test_loss= tf.keras.metrics.Mean(name= 'test_loss')
test_accuracy= tf.keras.metrics.SparseCategoricalAccuracy(name= 'test_accuracy')
test_recall= tf.keras.metrics.Recall(name= 'test_recall')
test_precision= tf.keras.metrics.Precision(name= 'test_precision')


current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = '/content/drive/MyDrive/Kika/Modelos guardados/Metricas/' + current_time + '/train'
test_log_dir = '/content/drive/MyDrive/Kika/Modelos guardados/Metricas/' + current_time + '/test'

train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [ ]:
@tf.function
def train_step(x, y, y_n):
    
    with tf.GradientTape() as tape:
        predict= mi_modelo(x, training= True)
        loss= loss_fn(y, predict)

    grads= tape.gradient(loss, mi_modelo.trainable_variables)
    optimizer.apply_gradients(zip(grads, mi_modelo.trainable_variables))

    train_loss(loss)
    train_accuracy(y, predict)
    train_recall(y_n, predict)
    train_precision(y_n, predict)

@tf.function
def test_step(x, y, y_n):
    
    predictions= mi_modelo(x, training= False)
    loss= loss_fn(y, predictions)

    test_loss(loss)
    test_accuracy(y, predictions)
    test_recall(y_n, predictions)
    test_precision(y_n, predictions)

In [ ]:
epocas= 150
loss_best= 1500

for epoca in range(epocas):
  train_loss.reset_states()
  train_accuracy.reset_states()
  train_recall.reset_states()
  train_precision.reset_states()

  test_loss.reset_state()
  test_accuracy.reset_state()
  test_recall.reset_states()
  test_precision.reset_states()

  for t in range(int(no_ent/batch_size_t)):
    images, labels= train_gen.dar_datos(input_img_paths, target_img_paths)
    y= tf.keras.utils.to_categorical(labels, num_classes= 2)
    train_step(images, labels, y)
  with train_summary_writer.as_default():
    tf.summary.scalar('Loss', train_loss.result(), step=epoca)
    tf.summary.scalar('Accuracy', train_accuracy.result(), step=epoca)
    tf.summary.scalar('Recall', train_recall.result(), step=epoca)
    tf.summary.scalar('Presicion', train_precision.result(), step=epoca)        

  for t in range(int(no_val/batch_size_t)):
    test_im, test_label= val_gen.dar_datos(val_input_paths, val_target_paths)
    y= tf.keras.utils.to_categorical(test_label, num_classes= 2)
    test_step(test_im, test_label, y)
  with test_summary_writer.as_default():
    tf.summary.scalar('Loss_t', test_loss.result(), step=epoca)
    tf.summary.scalar('Accuracy_t', test_accuracy.result(), step=epoca)
    tf.summary.scalar('Recall_t', test_recall.result(), step=epoca)
    tf.summary.scalar('Presicion_t', test_precision.result(), step=epoca) 
  
  if test_loss.result() < loss_best:
    loss_best= test_loss.result()/1
    mi_modelo.save_weights('/content/drive/MyDrive/Kika/Modelos guardados/Modelos/UNET-CBAM/UNET-CBAM')
    mi_modelo.save('/content/drive/MyDrive/Kika/Modelos guardados/Modelos/UNET-CBAM/UNET-CBAM.h5py')

  print('-------------------------------------------------\n')
  print(
      f'Epoca {epoca+ 1}, '
      f'Loss: {train_loss.result():.3f}, '
      f'Accuracy: {train_accuracy.result()* 100:.3f}, '
      f'Recall: {train_recall.result()* 100:.3f}, '
      f'Precision: {train_precision.result()* 100:.3f} '
      )
      
  print(
      f'Loss_test: {test_loss.result():.3f}, '
      f'Accuracy_test: {test_accuracy.result()* 100:.3f}, '
      f'Recall_test: {test_recall.result()* 100:.3f}, '
      f'Precision_test: {test_precision.result()* 100:.3f} '
  )
  print('\n-------------------------------------------------')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Kika/Modelos guardados/Modelos/UNET-CBAM/UNET-CBAM.h5py/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Kika/Modelos guardados/Modelos/UNET-CBAM/UNET-CBAM.h5py/assets


-------------------------------------------------

Epoca 1, Loss: 0.134, Accuracy: 95.403, Recall: 95.403, Precision: 95.403 
Loss_test: 0.153, Accuracy_test: 94.421, Recall_test: 94.421, Precision_test: 94.421 

-------------------------------------------------


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Kika/Modelos guardados/Modelos/UNET-CBAM/UNET-CBAM.h5py/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Kika/Modelos guardados/Modelos/UNET-CBAM/UNET-CBAM.h5py/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Kika/Modelos guardados/Modelos/UNET-CBAM/UNET-CBAM.h5py/assets


-------------------------------------------------

Epoca 12, Loss: 0.076, Accuracy: 97.015, Recall: 97.015, Precision: 97.015 
Loss_test: 0.076, Accuracy_test: 97.036, Recall_test: 97.036, Precision_test: 97.036 

-------------------------------------------------
-------------------------------------------------

Epoca 13, Loss: 0.075, Accuracy: 97.031, Recall: 97.031, Precision: 97.031 
Loss_test: 0.077, Accuracy_test: 96.998, Recall_test: 96.998, Precision_test: 96.998 

-------------------------------------------------


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Kika/Modelos guardados/Modelos/UNET-CBAM/UNET-CBAM.h5py/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Kika/Modelos guardados/Modelos/UNET-CBAM/UNET-CBAM.h5py/assets


-------------------------------------------------

Epoca 14, Loss: 0.075, Accuracy: 97.047, Recall: 97.046, Precision: 97.046 
Loss_test: 0.075, Accuracy_test: 97.074, Recall_test: 97.074, Precision_test: 97.074 

-------------------------------------------------
